# Preprocess the daily weather data

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import os
from pyspark.sql.functions import col, unix_timestamp, date_format, when, expr, round, hour
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


In [2]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 ")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

23/08/20 17:56:16 WARN Utils: Your hostname, LAPTOP-D9335T9D resolves to a loopback address: 127.0.1.1; using 192.168.0.77 instead (on interface wifi0)
23/08/20 17:56:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/20 17:56:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Reading data all weather data from january to June

In [3]:
sdf = spark.read.csv("../data/landing/weather_data/*", header=True, inferSchema=True)


In [4]:
sdf

name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
New york,2018-03-24 00:00:00,5.0,2.4,-4.4,50.44,0.0,0,null,0.0,0.73,null,11.2,241.0,1018.2,97.9,16.0,0.0,0.0,0,null,Overcast,cloudy,"72505394728,72055..."
New york,2018-03-24 01:00:00,4.3,2.5,-5.1,50.49,0.0,0,null,0.0,0.68,null,7.4,352.0,1018.4,97.9,16.0,0.0,0.0,0,null,Overcast,cloudy,"72505394728,72055..."
New york,2018-03-24 02:00:00,3.8,1.0,-5.1,52.22,0.0,0,null,0.0,0.62,42.5,11.4,312.0,1018.6,56.7,16.0,0.0,0.0,0,null,Partially cloudy,partly-cloudy-night,"72505394728,72055..."
New york,2018-03-24 03:00:00,3.3,0.4,-5.1,54.19,0.0,0,null,0.0,0.57,37.1,11.1,241.0,1018.5,88.7,16.0,0.0,0.0,0,null,Partially cloudy,partly-cloudy-night,"72505394728,72055..."
New york,2018-03-24 04:00:00,3.3,0.8,-5.1,54.19,0.0,0,null,0.0,0.51,null,9.4,305.0,1018.2,88.7,16.0,0.0,0.0,0,null,Partially cloudy,partly-cloudy-night,"72505394728,72055..."
New york,2018-03-24 05:00:00,2.8,0.2,-5.7,53.48,0.0,0,null,0.0,0.45,null,9.6,231.0,1018.2,4.4,16.0,0.0,0.0,0,null,Clear,clear-night,"72505394728,72055..."
New york,2018-03-24 06:00:00,2.8,2.8,-5.7,53.48,0.0,0,null,0.0,0.4,null,0.6,354.0,1018.5,4.4,16.0,0.9,0.0,0,null,Clear,clear-night,"72505394728,72055..."
New york,2018-03-24 07:00:00,2.2,-0.1,-6.2,53.92,0.0,0,null,0.0,0.34,null,7.7,320.0,1019.1,4.4,16.0,10.0,0.0,0,null,Clear,clear-day,"72505394728,72055..."
New york,2018-03-24 08:00:00,2.2,-0.6,-6.2,53.92,0.0,0,null,0.0,0.29,null,9.4,300.0,1019.6,4.4,16.0,53.0,0.2,1,null,Clear,clear-day,"72505394728,72055..."
New york,2018-03-24 09:00:00,4.3,2.3,-6.2,46.48,0.0,0,null,0.0,0.0,null,7.9,304.0,1019.5,8.8,16.0,258.0,0.9,3,null,Clear,clear-day,"72505394728,72055..."


### Filter our unnecessary feature and save it in raw data folder

In [5]:
columns_to_keep = ["datetime", "temp", 'feelslike', "snow","windspeed", 'cloudcover', "humidity", "sealevelpressure", "conditions"]
sdf = sdf.select(*columns_to_keep)

# save to raw 
output_folder = "../data/raw/weather_data/"  # Replace with the desired output folder
sdf.write.parquet(output_folder, mode="overwrite")

23/08/20 17:56:25 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


In [6]:
sdf = spark.read.parquet("../data/raw/weather_data/*")
sdf

datetime,temp,feelslike,snow,windspeed,cloudcover,humidity,sealevelpressure,conditions
2018-03-24 00:00:00,5.0,2.4,0.0,11.2,97.9,50.44,1018.2,Overcast
2018-03-24 01:00:00,4.3,2.5,0.0,7.4,97.9,50.49,1018.4,Overcast
2018-03-24 02:00:00,3.8,1.0,0.0,11.4,56.7,52.22,1018.6,Partially cloudy
2018-03-24 03:00:00,3.3,0.4,0.0,11.1,88.7,54.19,1018.5,Partially cloudy
2018-03-24 04:00:00,3.3,0.8,0.0,9.4,88.7,54.19,1018.2,Partially cloudy
2018-03-24 05:00:00,2.8,0.2,0.0,9.6,4.4,53.48,1018.2,Clear
2018-03-24 06:00:00,2.8,2.8,0.0,0.6,4.4,53.48,1018.5,Clear
2018-03-24 07:00:00,2.2,-0.1,0.0,7.7,4.4,53.92,1019.1,Clear
2018-03-24 08:00:00,2.2,-0.6,0.0,9.4,4.4,53.92,1019.6,Clear
2018-03-24 09:00:00,4.3,2.3,0.0,7.9,8.8,46.48,1019.5,Clear


## Now data cleaning

In [7]:
# snow depth cannot be negative 
sdf = sdf.filter(F.col("snow") >= 0) 
# wind speed cannot be negative 
sdf = sdf.filter(F.col("windspeed") >= 0) 
# assume temperate of a day in new york in 2018 does not 
# exceed the maximum temperature and minimum temperature record in US
sdf = sdf.filter(F.col("temp") < 56) 
sdf = sdf.filter(F.col("temp") > -62)
#remove any record with null 
sdf = sdf.na.drop()

# create a new feature to store the hour time of the day
sdf = sdf.withColumn("hour_of_the_day", hour(col("datetime")))
sdf = sdf.withColumn("date", date_format(col("datetime"), "yyyy-MM-dd"))

# Extract day of the week from pickup_time (Like Monday, Tuesday.....etc)
sdf = sdf.withColumn("what_day", date_format(col("date"), "EEEE"))
# Create a new column for weekday or weekend from what day 
sdf = sdf.withColumn("day_of_the_week", when(col("what_day").isin("Saturday", "Sunday"), "Weekend").otherwise("Weekday"))

# Drop the original "datetime" column as we will focuse on hourly analysi
sdf = sdf.drop("datetime", "what_day")

Save to curated data 

In [8]:
sdf

temp,feelslike,snow,windspeed,cloudcover,humidity,sealevelpressure,conditions,hour_of_the_day,date,day_of_the_week
5.0,2.4,0.0,11.2,97.9,50.44,1018.2,Overcast,0,2018-03-24,Weekend
4.3,2.5,0.0,7.4,97.9,50.49,1018.4,Overcast,1,2018-03-24,Weekend
3.8,1.0,0.0,11.4,56.7,52.22,1018.6,Partially cloudy,2,2018-03-24,Weekend
3.3,0.4,0.0,11.1,88.7,54.19,1018.5,Partially cloudy,3,2018-03-24,Weekend
3.3,0.8,0.0,9.4,88.7,54.19,1018.2,Partially cloudy,4,2018-03-24,Weekend
2.8,0.2,0.0,9.6,4.4,53.48,1018.2,Clear,5,2018-03-24,Weekend
2.8,2.8,0.0,0.6,4.4,53.48,1018.5,Clear,6,2018-03-24,Weekend
2.2,-0.1,0.0,7.7,4.4,53.92,1019.1,Clear,7,2018-03-24,Weekend
2.2,-0.6,0.0,9.4,4.4,53.92,1019.6,Clear,8,2018-03-24,Weekend
4.3,2.3,0.0,7.9,8.8,46.48,1019.5,Clear,9,2018-03-24,Weekend


In [9]:
# save to curated 
output= "../data/curated/weather_data/"  # Replace with the desired output folder
sdf.write.parquet(output, mode="overwrite")

23/08/20 17:56:28 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
